In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('reviews_cleaned.csv')
df.head()

,Company,Username,City,State,Date,Rating,Review,Distance,Cost,Cost.per.Mile,Sqft.Moved
0,Ryder,James W,Ocala,FL,2018-04-01,5,"I'd use them again, no problem.",48.0,142.0,2.96,NaN
1,Ryder,Jeanne M,Pittsburgh,PA,2017-11-02,5,they were fast and careful that nothing broke ...,25.0,1116.0,44.64,1000.0
2,Ryder,Dean G,Cary,NC,2017-04-21,4,"The truck was fine, but the training video bef...",NaN,NaN,NaN,NaN
3,Ryder,Michael A,Azusa,CA,2016-10-27,5,Great company!,NaN,NaN,NaN,NaN
4,Ryder,Tracy F,Rochester,NY,2016-01-19,5,First time using and would use again.,2400.0,2400.0,1.00,3000.0


In [3]:
df = df[['Company', 'City', 'State', 'Distance', 'Sqft.Moved']]

In [4]:
df.sample(10)

,Company,City,State,Distance,Sqft.Moved
4239,U-Haul,Morris Plains,NJ,70.0,400.0
3340,U-Haul,Colden,NY,15.0,1000.0
703,Budget,Miamisburg,OH,550.0,1000.0
49,Budget,Lawrenceville,GA,25.0,3500.0
1129,Penske,Acworth,GA,60.0,1300.0
4744,U-Haul,Oxnard,CA,30.0,1000.0
313,Budget,Atlantic Beach,FL,450.0,1200.0
220,Budget,Hammonton,NJ,10.0,1600.0
2301,Penske,Chandler,AZ,1200.0,4000.0
880,Budget,Lebanon,NJ,10.0,3000.0


In [5]:
df['City'] = df['City'].apply(lambda x: ' '.join(x.title() for x in x.split()))

In [6]:
import re
df['City'] = df['City'].apply(lambda x: re.sub('[^\w\s]', '', x))

In [9]:
df.shape

(5197, 5)

In [11]:
df = df.drop_duplicates()
df.shape

(5140, 5)

In [7]:
df_geocoder = pd.read_csv('geocoder.csv')
df_geocoder.head()

,City,State,latitude,longitude
0,Alabaster,AL,33.244281,-86.816377
1,Birmingham,AL,33.520682,-86.802433
2,Cullman,AL,34.133533,-86.877927
3,Daphne,AL,30.603525,-87.903605
4,Decatur,AL,34.661784,-87.092517


In [13]:
df_map = pd.merge(df, df_geocoder, how = 'left', on = ['City', 'State'])
df_map.head()

,Company,City,State,Distance,Sqft.Moved,latitude,longitude
0,Ryder,Ocala,FL,48.0,NaN,29.187199,-82.140092
1,Ryder,Pittsburgh,PA,25.0,1000.0,40.441694,-79.990086
2,Ryder,Cary,NC,NaN,NaN,35.788297,-78.781193
3,Ryder,Azusa,CA,NaN,NaN,34.133875,-117.905605
4,Ryder,Rochester,NY,2400.0,3000.0,43.157285,-77.615214


In [14]:
df_map.tail()

,Company,City,State,Distance,Sqft.Moved,latitude,longitude
5135,U-Haul,Little Rock,AR,NaN,NaN,34.746481,-92.289595
5136,U-Haul,Evergreen,CO,NaN,NaN,39.636164,-105.321458
5137,U-Haul,North Richland Hills,TX,NaN,NaN,32.834295,-97.228903
5138,U-Haul,Davenport,FL,NaN,NaN,28.161405,-81.601742
5139,U-Haul,Simi Valley,CA,NaN,NaN,34.269447,-118.781482


In [15]:
df_map.sample(5)

,Company,City,State,Distance,Sqft.Moved,latitude,longitude
2450,Penske,Fair Lawn,NJ,NaN,NaN,40.940376,-74.131810
953,Budget,New York,NY,85.0,400.0,40.741236,-73.356691
3345,U-Haul,Apex,NC,5.0,1900.0,35.732535,-78.850552
524,Budget,Kansas City,MO,189.0,900.0,39.084469,-94.563030
110,Budget,State College,PA,200.0,1200.0,40.787912,-77.853886


In [17]:
import numpy as np

In [18]:
np.sum(df_map.isnull(), axis = 0)

Company         0
City            0
State           0
Distance      490
Sqft.Moved    987
latitude       40
longitude      40
dtype: int64

In [19]:
df_map.shape

(5140, 7)

In [20]:
df_map = df_map[df_map['latitude'].notnull()]
df_map.shape

(5100, 7)

In [21]:
import plotly.plotly as py

In [22]:
df_map_count = df_map.groupby(['City', 'State', 'latitude', 'longitude']).size().reset_index()
df_map_count.columns = ['City', 'State', 'latitude', 'longitude', 'count']
df_map_count.head(5)

,City,State,latitude,longitude,count
0,Aberdeen,MD,39.509556,-76.164120,2
1,Abingdon,MD,39.462331,-76.279124,1
2,Abita Springs,LA,30.478526,-90.037576,1
3,Acworth,GA,34.065933,-84.676880,4
4,Addison,TX,32.960431,-96.830260,1


In [23]:
df_map_count['text'] = df_map_count['City'] + ', ' + df_map_count['State']
df_map_count.head(5)

,City,State,latitude,longitude,count,text
0,Aberdeen,MD,39.509556,-76.164120,2,"Aberdeen, MD"
1,Abingdon,MD,39.462331,-76.279124,1,"Abingdon, MD"
2,Abita Springs,LA,30.478526,-90.037576,1,"Abita Springs, LA"
3,Acworth,GA,34.065933,-84.676880,4,"Acworth, GA"
4,Addison,TX,32.960431,-96.830260,1,"Addison, TX"


In [24]:
cities = []
city = dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lat = df_map_count['latitude'], 
    lon = df_map_count['longitude'], 
    text = df_map_count['text'],
    marker = dict(
        size = df_map_count['count'],
        color = 'rgb(0,116,217)',
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'),
    name = '{0} - {1}'.format(0, df_map_count.shape[1])
)
cities.append(city)

In [25]:
layout = dict(
    title = 'All Reviewer Locations',
    showlegend = True,
    geo = dict(
        scope='usa',
        projection=dict(type='albers usa'),
        showland = True,
        landcolor = 'rgb(217, 217, 217)',
        subunitwidth=1,
        countrywidth=1,
        subunitcolor="rgb(255, 255, 255)",
        countrycolor="rgb(255, 255, 255)"
    ),
)

In [26]:
# plotly username: plotlylearner, gmail
# password: 8+eng+$*
# regenerate the key: https://plot.ly/settings/api#/
import plotly
plotly.tools.set_credentials_file(username='plotlylearner', api_key = 'Or2tF6uyx8YdAKkRR0nY' )

In [27]:
fig = dict(data = cities, layout = layout)
py.iplot(fig, validate=False, filename='d3-bubble-map-all-reviews')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~plotlylearner/0 or inside your plot.ly account where it is named 'd3-bubble-map-all-reviews'


In [28]:
df_map_campany = df_map.groupby(['Company', 'City', 'State', 'latitude', 'longitude']).size().reset_index()
df_map_campany.columns = ['Company', 'City', 'State', 'latitude', 'longitude', 'count']
df_map_campany.head(5)

,Company,City,State,latitude,longitude,count
0,Budget,Aberdeen,MD,39.509556,-76.164120,1
1,Budget,Aiken,SC,33.559859,-81.721952,1
2,Budget,Alabaster,AL,33.244281,-86.816377,1
3,Budget,Alameda,CA,49.266667,-102.283333,1
4,Budget,Albemarle,NC,35.350143,-80.200058,1


In [30]:
df_map_campany['text'] = df_map_campany['City'] + ', ' + df_map_campany['State']
df_map_campany.head(5)

,Company,City,State,latitude,longitude,count,text
0,Budget,Aberdeen,MD,39.509556,-76.164120,1,"Aberdeen, MD"
1,Budget,Aiken,SC,33.559859,-81.721952,1,"Aiken, SC"
2,Budget,Alabaster,AL,33.244281,-86.816377,1,"Alabaster, AL"
3,Budget,Alameda,CA,49.266667,-102.283333,1,"Alameda, CA"
4,Budget,Albemarle,NC,35.350143,-80.200058,1,"Albemarle, NC"


In [31]:
df_map_campany.shape

(3049, 7)

In [32]:
df_map_campany[df_map_campany['Company'] == 'Budget']['Company'].count()

665

In [33]:
df_map_campany[df_map_campany['Company'] == 'Enterprise']['Company'].count()

17

In [34]:
df_map_campany[df_map_campany['Company'] == 'Penske']['Company'].count()

884

In [35]:
df_map_campany[df_map_campany['Company'] == 'Ryder']['Company'].count()

16

In [36]:
df_map_campany[df_map_campany['Company'] == 'U-Haul']['Company'].count()

1467

In [37]:
df_map_campany.shape

(3049, 7)

In [41]:
print(df_map_campany.iloc[664,0], df_map_campany.iloc[665,0])

Budget Enterprise


In [42]:
print(df_map_campany.iloc[681,0], df_map_campany.iloc[682,0])

Enterprise Penske


In [46]:
print(df_map_campany.iloc[1565,0], df_map_campany.iloc[1566,0])

Penske Ryder


In [48]:
print(df_map_campany.iloc[1581,0], df_map_campany.iloc[1582,0])

Ryder U-Haul


In [50]:
df_map_campany.iloc[3048,0]

'U-Haul'

In [54]:
# company: budget, enterprise, penske, ryder, uhaul
limits = [(0,665),(665,682),(682,1566),(1566,1582),(1582,3049)]
colors = ["rgb(64,66,122)","rgb(29,121,81)","rgb(206,174,57)","rgb(166,24,42)","rgb(247,87,63)"]

In [55]:
cities = []
for i in range(len(limits)):
    lim = limits[i]
    df_sub = df_map_campany[lim[0]:lim[1]]
    city = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lat = df_map_campany['latitude'], 
        lon = df_map_campany['longitude'], 
        text = df_map_campany['text'],
        marker = dict(
            size = df_map_campany['count']*8,
            color = colors[i],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1])
    )
    cities.append(city)

In [56]:
layout = dict(
    title = 'Company Distribution',
    showlegend = True,
    geo = dict(
        scope='usa',
        projection=dict(type='albers usa'),
        showland = True,
        landcolor = 'rgb(217, 217, 217)',
        subunitwidth=1,
        countrywidth=1,
        subunitcolor="rgb(255, 255, 255)",
        countrycolor="rgb(255, 255, 255)"
    ),
)

In [58]:
fig = dict(data = cities, layout = layout)
py.iplot(fig, validate=False, filename='d3-bubble-map-companies')

# Budget

In [ ]:
# company: budget, enterprise, penske, ryder, uhaul
# Budget, Enterprise, Penske, Ryder, U-Haul
# limits = [(0,665),(665,682),(682,1566),(1566,1582),(1582,3049)]
# colors = ["rgb(64,66,122)","rgb(29,121,81)","rgb(206,174,57)","rgb(166,24,42)","rgb(247,87,63)"]

In [59]:
df_sub = df_map_campany[0:665]
df_sub.shape

(665, 7)

In [60]:
df_sub.sample(5)

,Company,City,State,latitude,longitude,count,text
629,Budget,Waukegan,IL,42.363633,-87.844794,1,"Waukegan, IL"
148,Budget,Diamond Bar,CA,34.028623,-117.810337,1,"Diamond Bar, CA"
596,Budget,Tampa,FL,27.947759,-82.458444,6,"Tampa, FL"
20,Budget,Antioch,CA,38.004921,-121.805789,1,"Antioch, CA"
331,Budget,Los Angeles,CA,34.054935,-118.244476,4,"Los Angeles, CA"


In [69]:
cities = []
city = dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lat = df_sub['latitude'], 
    lon = df_sub['longitude'], 
    text = df_sub['text'],
    marker = dict(
        size = df_sub['count']*8,
        color = 'rgb(64,66,122)',
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'),
    name = '{0} - {1}'.format(0, df_sub.shape[1])
)
cities.append(city)

In [70]:
layout = dict(
    title = 'Budget Locations',
    showlegend = True,
    geo = dict(
        scope='usa',
        projection=dict(type='albers usa'),
        showland = True,
        landcolor = 'rgb(217,217,217)',
        subunitwidth=1,
        countrywidth=1,
        subunitcolor="rgb(255, 255, 255)",
        countrycolor="rgb(255, 255, 255)"
    ),
)

In [71]:
fig = dict(data = cities, layout = layout)
py.iplot(fig, validate=False, filename='d3-bubble-map-budget')

# Enterprise

In [ ]:
# company: budget, enterprise, penske, ryder, uhaul
# Budget, Enterprise, Penske, Ryder, U-Haul
# limits = [(0,665),(665,682),(682,1566),(1566,1582),(1582,3049)]
# colors = ["rgb(64,66,122)","rgb(29,121,81)","rgb(206,174,57)","rgb(166,24,42)","rgb(247,87,63)"]

In [72]:
df_sub = df_map_campany[665:682]
df_sub.shape

(17, 7)

In [73]:
df_sub.sample(5)

,Company,City,State,latitude,longitude,count,text
679,Enterprise,Romeoville,IL,41.647531,-88.089506,1,"Romeoville, IL"
680,Enterprise,Virginia Beach,VA,36.852984,-75.977418,1,"Virginia Beach, VA"
671,Enterprise,Davis,CA,53.110333,-105.656724,1,"Davis, CA"
677,Enterprise,Richmond Hill,GA,31.938265,-81.303443,1,"Richmond Hill, GA"
676,Enterprise,Raleigh,NC,35.780398,-78.639099,1,"Raleigh, NC"


In [79]:
cities = []
city = dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lat = df_sub['latitude'], 
    lon = df_sub['longitude'], 
    text = df_sub['text'],
    marker = dict(
        size = df_sub['count']*30,
        color = 'rgb(29,121,81)',
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'),
    name = '{0} - {1}'.format(0, df_sub.shape[1])
)
cities.append(city)

In [75]:
layout = dict(
    title = 'Enterprise Locations',
    showlegend = True,
    geo = dict(
        scope='usa',
        projection=dict(type='albers usa'),
        showland = True,
        landcolor = 'rgb(217,217,217)',
        subunitwidth=1,
        countrywidth=1,
        subunitcolor="rgb(255, 255, 255)",
        countrycolor="rgb(255, 255, 255)"
    ),
)

In [80]:
fig = dict(data = cities, layout = layout)
py.iplot(fig, validate=False, filename='d3-bubble-map-enterprise')

# Penske

In [ ]:
# company: budget, enterprise, penske, ryder, uhaul
# Budget, Enterprise, Penske, Ryder, U-Haul
# limits = [(0,665),(665,682),(682,1566),(1566,1582),(1582,3049)]
# colors = ["rgb(64,66,122)","rgb(29,121,81)","rgb(206,174,57)","rgb(166,24,42)","rgb(247,87,63)"]

In [81]:
df_sub = df_map_campany[682:1566]
df_sub.shape

(884, 7)

In [82]:
df_sub.sample(5)

,Company,City,State,latitude,longitude,count,text
875,Penske,Darien,CT,41.078708,-73.469287,1,"Darien, CT"
806,Penske,Champaign,IL,40.116421,-88.243383,1,"Champaign, IL"
898,Penske,Dundee,IL,56.460594,-2.970190,1,"Dundee, IL"
1299,Penske,Paso Robles,CA,35.626765,-120.691246,1,"Paso Robles, CA"
1390,Penske,San Francisco,CA,45.442354,-73.437309,2,"San Francisco, CA"


In [83]:
cities = []
city = dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lat = df_sub['latitude'], 
    lon = df_sub['longitude'], 
    text = df_sub['text'],
    marker = dict(
        size = df_sub['count']*8,
        color = 'rgb(206,174,57)',
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'),
    name = '{0} - {1}'.format(0, df_sub.shape[1])
)
cities.append(city)

In [84]:
layout = dict(
    title = 'Penske Locations',
    showlegend = True,
    geo = dict(
        scope='usa',
        projection=dict(type='albers usa'),
        showland = True,
        landcolor = 'rgb(217,217,217)',
        subunitwidth=1,
        countrywidth=1,
        subunitcolor="rgb(255, 255, 255)",
        countrycolor="rgb(255, 255, 255)"
    ),
)

In [85]:
fig = dict(data = cities, layout = layout)
py.iplot(fig, validate=False, filename='d3-bubble-map-penske')

# Ryder

In [ ]:
# company: budget, enterprise, penske, ryder, uhaul
# Budget, Enterprise, Penske, Ryder, U-Haul
# limits = [(0,665),(665,682),(682,1566),(1566,1582),(1582,3049)]
# colors = ["rgb(64,66,122)","rgb(29,121,81)","rgb(206,174,57)","rgb(166,24,42)","rgb(247,87,63)"]

In [86]:
df_sub = df_map_campany[1566:1582]
df_sub.shape

(16, 7)

In [87]:
df_sub.sample(5)

,Company,City,State,latitude,longitude,count,text
1575,Ryder,Ocala,FL,29.187199,-82.140092,1,"Ocala, FL"
1572,Ryder,Crowley,TX,32.579025,-97.362517,1,"Crowley, TX"
1581,Ryder,Rochester,NY,43.157285,-77.615214,1,"Rochester, NY"
1573,Ryder,Dallas,TX,32.776272,-96.796856,1,"Dallas, TX"
1577,Ryder,Pittsburgh,PA,40.441694,-79.990086,1,"Pittsburgh, PA"


In [89]:
cities = []
city = dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lat = df_sub['latitude'], 
    lon = df_sub['longitude'], 
    text = df_sub['text'],
    marker = dict(
        size = df_sub['count']*30,
        color = 'rgb(166,24,42)',
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'),
    name = '{0} - {1}'.format(0, df_sub.shape[1])
)
cities.append(city)

In [90]:
layout = dict(
    title = 'Ryder Locations',
    showlegend = True,
    geo = dict(
        scope='usa',
        projection=dict(type='albers usa'),
        showland = True,
        landcolor = 'rgb(217,217,217)',
        subunitwidth=1,
        countrywidth=1,
        subunitcolor="rgb(255, 255, 255)",
        countrycolor="rgb(255, 255, 255)"
    ),
)

In [91]:
fig = dict(data = cities, layout = layout)
py.iplot(fig, validate=False, filename='d3-bubble-map-ryder')

# U-Haul

In [ ]:
# company: budget, enterprise, penske, ryder, uhaul
# Budget, Enterprise, Penske, Ryder, U-Haul
# limits = [(0,665),(665,682),(682,1566),(1566,1582),(1582,3049)]
# colors = ["rgb(64,66,122)","rgb(29,121,81)","rgb(206,174,57)","rgb(166,24,42)","rgb(247,87,63)"]

In [92]:
df_sub = df_map_campany[1582:]
df_sub.shape

(1467, 7)

In [93]:
df_sub.sample(5)

,Company,City,State,latitude,longitude,count,text
2495,U-Haul,Newton,MA,42.337041,-71.209221,1,"Newton, MA"
1608,U-Haul,Antioch,CA,38.004921,-121.805789,2,"Antioch, CA"
2531,U-Haul,Oaks,PA,40.131772,-75.459632,1,"Oaks, PA"
2632,U-Haul,Portland,OR,45.520247,-122.674195,2,"Portland, OR"
2060,U-Haul,Friendswood,TX,29.529400,-95.201045,1,"Friendswood, TX"


In [94]:
cities = []
city = dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lat = df_sub['latitude'], 
    lon = df_sub['longitude'], 
    text = df_sub['text'],
    marker = dict(
        size = df_sub['count']*8,
        color = 'rgb(247,87,63)',
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'),
    name = '{0} - {1}'.format(0, df_sub.shape[1])
)
cities.append(city)

In [95]:
layout = dict(
    title = 'U-Haul Locations',
    showlegend = True,
    geo = dict(
        scope='usa',
        projection=dict(type='albers usa'),
        showland = True,
        landcolor = 'rgb(217,217,217)',
        subunitwidth=1,
        countrywidth=1,
        subunitcolor="rgb(255, 255, 255)",
        countrycolor="rgb(255, 255, 255)"
    ),
)

In [96]:
fig = dict(data = cities, layout = layout)
py.iplot(fig, validate=False, filename='d3-bubble-map-uhaul')

In [71]:
df_geo = df[['City', 'State']]
df_geo.head()

,City,State
0,Ocala,FL
1,Pittsburgh,PA
2,Cary,NC
3,Azusa,CA
4,Rochester,NY


In [25]:
df_geo.shape

(5197, 2)

In [72]:
df_geo = df_geo.drop_duplicates()

In [27]:
df_geo.shape

(2246, 2)

In [9]:
from geopy import geocoders
from geopy.geocoders import Nominatim
# geopy username: geopylearner, gmail
# geopy password: 8+eng+$*

In [10]:
geolocator = Nominatim(user_agent="geopylearner")

In [12]:
location = geolocator.geocode("New York, NY")
print(location.latitude, location.longitude)
location = geolocator.geocode("Houston, TX")
print(location.latitude, location.longitude)

40.74123595 -73.356691165361
29.7589382 -95.3676974


In [13]:
type(location)

geopy.location.Location

In [14]:
type(location.longitude)

float

In [15]:
a, b = location.latitude, location.longitude
a, b

(29.7589382, -95.3676974)

In [17]:
location = geolocator.geocode('Ortley Beach Toms River Twpshp, NJ')
# print(location.latitude, location.longitude)
location == None

True

In [11]:
import numpy as np

In [12]:
from geopy.exc import GeocoderTimedOut

In [73]:
df_geo = df_geo.sort_values(['State', 'City'])
df_geo['latitude'] = 0.0
df_geo['longitude'] = 0.0
df_geo = df_geo.reset_index(drop=True)
df_geo.head(5)

,City,State,latitude,longitude
0,Alabaster,AL,0.0,0.0
1,Birmingham,AL,0.0,0.0
2,Cullman,AL,0.0,0.0
3,Daphne,AL,0.0,0.0
4,Decatur,AL,0.0,0.0


In [74]:
df_geo.iloc[1:3]

,City,State,latitude,longitude
1,Birmingham,AL,0.0,0.0
2,Cullman,AL,0.0,0.0


In [75]:
def do_geocode(address):
    try:
        geolocator = Nominatim(user_agent="geopylearner")
        return geolocator.geocode(address)
    except GeocoderTimedOut as e:
        return do_geocode(address)

def find_geocoder_func(x):
    addr = x['City'] + ', ' + x['State']
    print(addr)
    location = do_geocode(addr)
    if location == None:
        x['latitude'] = np.nan
        x['longitude'] = np.nan
    else:
        x['latitude'] = location.latitude
        x['longitude'] = location.longitude
    return x

In [308]:
df_geo.iloc[2240:2246] = df_geo.iloc[2240:2246].apply(find_geocoder_func, axis = 1)

Martinsburg, WV
Martinsburg, WV
Ranson, WV
Saint Albans, WV
Shepherdstown, WV
Centennial, WY
Cheyenne, WY


In [305]:
df_geo.iloc[2238:2242]

,City,State,latitude,longitude
2238,Harpers Ferry,WV,39.323418,-77.731977
2239,Huntington,WV,38.419250,-82.445154
2240,Martinsburg,WV,0.000000,0.000000
2241,Ranson,WV,0.000000,0.000000


In [309]:
df_geo.tail(10)

,City,State,latitude,longitude
2236,Charles Town,WV,39.289184,-77.859581
2237,Falling Waters,WV,39.559264,-77.890830
2238,Harpers Ferry,WV,39.323418,-77.731977
2239,Huntington,WV,38.419250,-82.445154
2240,Martinsburg,WV,39.456253,-77.963960
2241,Ranson,WV,39.295282,-77.860383
2242,Saint Albans,WV,38.385649,-81.836241
2243,Shepherdstown,WV,39.430100,-77.804161
2244,Centennial,WY,41.297896,-106.139369
2245,Cheyenne,WY,41.139981,-104.820246


A: AL AK AZ AR
C: CA CO CT
D: DE DC
F: FL
G: GA
H: HI
I: ID IL IN IA
K: KS KY
L: LA
M: ME MD MA MI MN MS MO MT
N: NE NV NH NJ NM NY NC ND
O: OH OK OR
P: PA
R: RI
S: SC SD
T: TN TX
U: UT
V: VT VA
W: WA WV WI WY

IPv4 Address. . . . . . . . . . . : 192.168.0.104
<br> Subnet Mask . . . . . . . . . . . : 255.255.255.0
<br> Default Gateway . . . . . . . . . : 192.168.0.1

In [24]:
df_geo[df_geo['State'].str.startswith('A')]['State'].count()

72

In [26]:
df_geo[df_geo['State'].str.startswith('C')]['State'].count()

306

In [27]:
df_geo[df_geo['State'].str.startswith('D')]['State'].count()

13

In [29]:
df_geo[df_geo['State'].str.startswith('F')]['State'].count()

159

In [30]:
df_geo[df_geo['State'].str.startswith('G')]['State'].count()

96

In [32]:
df_geo[df_geo['State'].str.startswith('I')]['State'].count()

155

In [34]:
df_geo[df_geo['State'].str.startswith('K')]['State'].count()

26

In [35]:
df_geo[df_geo['State'].str.startswith('L')]['State'].count()

19

In [36]:
df_geo[df_geo['State'].str.startswith('M')]['State'].count()

319

In [37]:
df_geo[df_geo['State'].str.startswith('N')]['State'].count()

336

In [38]:
df_geo[df_geo['State'].str.startswith('O')]['State'].count()

127

In [39]:
df_geo[df_geo['State'].str.startswith('P')]['State'].count()

135

In [41]:
df_geo[df_geo['State'].str.startswith('R')]['State'].count()

7

In [42]:
df_geo[df_geo['State'].str.startswith('S')]['State'].count()

56

In [43]:
df_geo[df_geo['State'].str.startswith('T')]['State'].count()

183

In [44]:
df_geo[df_geo['State'].str.startswith('U')]['State'].count()

19

In [45]:
df_geo[df_geo['State'].str.startswith('V')]['State'].count()

87

In [46]:
df_geo[df_geo['State'].str.startswith('W')]['State'].count()

131

In [19]:
df_geo = df_geo.apply(find_geocoder_func, axis = 1)

GeocoderQuotaExceeded: ('HTTP Error 429: Too Many Requests', 'occurred at index 2348')

In [311]:
df_geo['City'] = df_geo['City'].apply(lambda x: ' '.join(x.title() for x in x.split()))

In [312]:
df_geo.shape

(2246, 4)

In [313]:
df_geo = df_geo.drop_duplicates()

In [314]:
df_geo.shape

(2215, 4)

In [316]:
df_geo = df_geo.drop_duplicates()
df_geo.shape

(2215, 4)

In [317]:
df_geo.to_csv('geocoder.csv', index = False)